In [19]:
import pandas as pd

In [21]:
df=pd.read_csv('dataset-2.csv')
df.head(3)

,ï»¿id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0


In [23]:
df.rename(columns={df.columns[0]: 'id_start'}, inplace=True)

Question 09

In [33]:

def calculate_distance_matrix(df):

    distances = {}

    for index, row in df.iterrows():
        source = row['id_start']
        target = row['id_end']
        distance = row['distance']

        distances[(source, target)] = distance
        distances[(target, source)] = distance 
    unique_locations = sorted(list(set(df['id_start'].unique()) | set(df['id_end'].unique())))

    distance_matrix = pd.DataFrame(index=unique_locations, columns=unique_locations)
    distance_matrix = distance_matrix.fillna(0)

    for start_location in unique_locations:
        for end_location in unique_locations:
            if start_location == end_location:
                distance_matrix.loc[start_location, end_location] = 0
            else:            
                path_distance = 0
                try:
                    path_distance = distances[(start_location, end_location)]
                except KeyError:
                    for intermediate_location in unique_locations:
                        if intermediate_location != start_location and intermediate_location != end_location:
                            try:
                                distance1 = distances[(start_location, intermediate_location)]
                                distance2 = distances[(intermediate_location, end_location)]
                                if distance1 and distance2:
                                    path_distance = max(path_distance, distance1 + distance2)
                            except KeyError:
                                continue


                distance_matrix.loc[start_location, end_location] = path_distance

    return distance_matrix


distance_matrix = calculate_distance_matrix(df)
df2_r=pd.DataFrame(distance_matrix)
df2_r.head(5)


,1001400,1001402,1001404,1001406,1001408,1001410,1001412,1001414,1001416,1001418,...,1001462,1001464,1001466,1001468,1001470,1001472,1001488,1004354,1004355,1004356
1001400,0.0,9.7,29.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001402,9.7,0.0,20.2,36.2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001404,29.9,20.2,0.0,16.0,37.7,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001406,0.0,36.2,16.0,0.0,21.7,32.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001408,0.0,0.0,37.7,21.7,0.0,11.1,26.7,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Question 10

In [29]:
def unroll_distance_matrix(df):
    rows = []
    for id_start in df.index:
        for id_end in df.columns:
            if id_start != id_end:
                distance = df.loc[id_start, id_end]
                rows.append({'id_start': id_start, 'id_end': id_end, 'distance': distance})
    return pd.DataFrame(rows)

In [39]:
new_1=unroll_distance_matrix(df2_r)
new_1

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001400,1001404,29.9
2,1001400,1001406,0.0
3,1001400,1001408,0.0
4,1001400,1001410,0.0
...,...,...,...
1801,1004356,1001470,0.0
1802,1004356,1001472,0.0
1803,1004356,1001488,4.0
1804,1004356,1004354,2.0


Question 11

In [35]:
def find_ids_within_ten_percentage_threshold(df, reference_value):
  avg_distances = df.groupby('id_start')['distance'].mean()
  reference_avg_distance = avg_distances[reference_value]
  lower_bound = reference_avg_distance * 0.9
  upper_bound = reference_avg_distance * 1.1

  within_threshold_ids = sorted([
      id_start for id_start, avg_distance in avg_distances.items()
      if lower_bound <= avg_distance <= upper_bound
  ])

  return within_threshold_ids

In [41]:
find_ids_within_ten_percentage_threshold(new_1,1001400)

[1001400, 1001472]

In [45]:
def calculate_toll_rate(df):
  df['moto'] = df['distance'] * 0.8
  df['car'] = df['distance'] * 1.2
  df['rv'] = df['distance'] * 1.5
  df['bus'] = df['distance'] * 2.2
  df['truck'] = df['distance'] * 3.6
  return df

df_with_toll = calculate_toll_rate(df)

In [47]:
df_with_toll.sample(2)

,id_start,id_end,distance,moto,car,rv,bus,truck
9,1001418,1001420,12.9,10.32,15.48,19.35,28.38,46.44
27,1004355,1001444,0.7,0.56,0.84,1.05,1.54,2.52


In [49]:
from datetime import time


def calculate_time_based_toll_rates(df_11):
  df_time_based = df_11.copy()
  time_ranges = [
      (time(0, 0, 0), time(10, 0, 0), 0.8),  
      (time(10, 0, 0), time(18, 0, 0), 1.2),  
      (time(18, 0, 0), time(23, 59, 59), 0.8),  
  ]
  weekend_discount_factor = 0.7
  df_time_based['start_day'] = ''
  df_time_based['start_time'] = None
  df_time_based['end_day'] = ''
  df_time_based['end_time'] = None

  for index, row in df_time_based.iterrows():

    for day in range(7):
        weekday = (day)

        if weekday < 5:

          for start_time, end_time, discount_factor in time_ranges:

            df_time_based.loc[index,'start_day'] = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'][weekday]
            df_time_based.loc[index,'start_time'] = start_time
            df_time_based.loc[index,'end_day'] = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'][weekday]
            df_time_based.loc[index,'end_time'] = end_time

            for vehicle_type in ['moto', 'car', 'rv', 'bus', 'truck']:
                df_time_based.loc[index, vehicle_type] = df_time_based.loc[index, vehicle_type] * discount_factor
        else:

            df_time_based.loc[index,'start_day'] = ['Saturday', 'Sunday'][weekday-5]
            df_time_based.loc[index,'start_time'] = time(0, 0, 0)
            df_time_based.loc[index,'end_day'] = ['Saturday', 'Sunday'][weekday-5]
            df_time_based.loc[index,'end_time'] = time(23, 59, 59)


            for vehicle_type in ['moto', 'car', 'rv', 'bus', 'truck']:
                df_time_based.loc[index, vehicle_type] = df_time_based.loc[index, vehicle_type] * weekend_discount_factor
  return df_time_based

df_time_based_toll_rates=calculate_time_based_toll_rates(df_with_toll)
print(df_time_based_toll_rates.head(2))

   id_start   id_end  distance      moto       car        rv       bus  \
0   1001400  1001402       9.7  1.015930  1.523895  1.904869  2.793808   
1   1001402  1001404      20.2  2.115649  3.173473  3.966841  5.818034   

      truck start_day start_time end_day  end_time  
0  4.571686    Sunday   00:00:00  Sunday  23:59:59  
1  9.520419    Sunday   00:00:00  Sunday  23:59:59  


In [51]:
pd.DataFrame(df_time_based_toll_rates)

,id_start,id_end,distance,moto,car,rv,bus,truck,start_day,start_time,end_day,end_time
0,1001400,1001402,9.7,1.015930,1.523895,1.904869,2.793808,4.571686,Sunday,00:00:00,Sunday,23:59:59
1,1001402,1001404,20.2,2.115649,3.173473,3.966841,5.818034,9.520419,Sunday,00:00:00,Sunday,23:59:59
2,1001404,1001406,16.0,1.675761,2.513642,3.142052,4.608344,7.540926,Sunday,00:00:00,Sunday,23:59:59
3,1001406,1001408,21.7,2.272751,3.409127,4.261409,6.250066,10.227381,Sunday,00:00:00,Sunday,23:59:59
4,1001408,1001410,11.1,1.162559,1.743839,2.179799,3.197038,5.231517,Sunday,00:00:00,Sunday,23:59:59
5,1001410,1001412,15.6,1.633867,2.450801,3.063501,4.493135,7.352403,Sunday,00:00:00,Sunday,23:59:59
6,1001412,1001414,18.2,1.906178,2.859268,3.574085,5.241991,8.577803,Sunday,00:00:00,Sunday,23:59:59
7,1001414,1001416,13.2,1.382503,2.073755,2.592193,3.801883,6.221264,Sunday,00:00:00,Sunday,23:59:59
8,1001416,1001418,13.6,1.424397,2.136596,2.670745,3.917092,6.409787,Sunday,00:00:00,Sunday,23:59:59
9,1001418,1001420,12.9,1.351083,2.026624,2.533280,3.715477,6.079871,Sunday,00:00:00,Sunday,23:59:59
